# Célula 1: AGENTE01 — Agente de IA para Perguntas e Respostas

Este agente tem como objetivo responder a perguntas feitas pelo usuário com base em um conteúdo pré-carregado (como arquivos `.txt`, `.pdf` ou outros documentos). 

A ideia central é criar uma interface simples e funcional para interagir com um LLM (Large Language Model), utilizando ferramentas como:

- **LangChain**: para orquestração do agente e carregamento dos dados
- **Modelos Hugging Face**: para gerar respostas com base nos dados
- **Chroma**: para armazenar embeddings e fazer buscas semânticas

---

## Aplicações possíveis:
- Consulta a manuais técnicos
- Resumo de relatórios
- Atendimento automático baseado em documentos

---

## Etapas abordadas neste notebook:

**🔹 Célula 2 – Definição do texto base**
- Define manualmente o conteúdo de conhecimento.
- Utilizado como contexto para geração de embeddings e respostas.

**🔹 Célula 3 – Geração de embeddings com modelo local**
- Cria um documento LangChain a partir do texto.
- Divide o conteúdo em partes com o `CharacterTextSplitter`.
- Gera embeddings com modelo local `sentence-transformers/all-MiniLM-L6-v2`.
- Armazena os embeddings em uma base Chroma para consultas futuras.

**🔹 Célula 4 – Consulta semântica**
- Recebe uma pergunta do usuário via `input()`.
- Recupera os documentos mais relevantes da base vetorial.
- Exibe o conteúdo mais semelhante para uso como contexto.

**🔹 Célula 5 – Simulação automatizada**
- Contém uma pergunta fixa (sem input) para simular a consulta automática.
- Realiza busca semântica e exibe o conteúdo retornado.

**🔹 Célula 6 – Execução do agente com modelo generativo (FLAN-T5)**
- Usa `pipeline` com o modelo `google/flan-t5-base`.
- Monta o prompt com o texto base e a pergunta.
- Gera uma resposta textual com base no contexto.
- OBS: O modelo é treinado em inglês, mas pode responder em português.

**🔹 Célula 7 – Execução do agente com modelo generativo (mT5)**
- Usa o modelo `google/mt5-small`, com suporte multilíngue.
- Gera respostas mais completas em português, a partir do conteúdo carregado.

---


In [ ]:
"""
Célula 2: Bibliotecas principais do agente local
    Esta célula instala as bibliotecas necessárias para o agente local.
"""

executar_celula_5 = False

if executar_celula_5:
    # Bibliotecas principais do agente local
    !pip install -U langchain langchain-community chromadb tiktoken sentence-transformers transformers
    # Leitura de documentos (opcional)
    !pip install unstructured pdfminer.six python-docx
    # (Opcional) Interface interativa
    !pip install ipywidgets
    !pip install blobfile
    !pip install sentencepiece
    !pip install sacremoses

else:
    # código da célula 5
    print("Execução interrompida da célula...")


Execução interrompida da célula...


In [2]:
"""
Célula 3: Carregamento de conteúdo fixo e criação dos embeddings locais

Esta célula:
- Define um texto manual como base de conhecimento
- Separa o texto em pequenos blocos (chunking)
- Gera embeddings com modelo local HuggingFace (sentence-transformers)
- Armazena os embeddings em uma base vetorial local usando Chroma
"""

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document


# Texto fixo como exemplo de base de conhecimento
texto = """
A inteligência artificial (IA) é o ramo da ciência da computação que se dedica a criar sistemas capazes de executar tarefas que normalmente exigiriam inteligência humana.
Essas tarefas incluem reconhecimento de fala, tomada de decisões, tradução de idiomas e muito mais.
Os agentes de IA utilizam modelos treinados em grandes quantidades de dados para realizar inferências e responder a comandos.
"""

# Separar o texto em blocos menores para melhor indexação
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=20)
documentos = text_splitter.split_documents([Document(page_content=texto)])

# Gerar embeddings locais com modelo Hugging Face
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Criar e armazenar os vetores em uma base vetorial local com Chroma
vectorstore = Chroma.from_documents(documentos, embedding=embedding)

C:\Users\carlo\AppData\Local\Temp\ipykernel_15416\460049780.py:29: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [3]:
"""
Célula 4: Execução do agente de perguntas e respostas

Esta célula:
- Recebe uma pergunta do usuário
- Usa o modelo local 'deepset/roberta-base-squad2' via Transformers
- Retorna a resposta com base no texto carregado
- Modelo com resposta a perguntas (QA) local, roberta-base-squad2, RESPOSTAS GENERICAS
"""

from transformers import pipeline

# Pipeline de QA
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Texto base como contexto
contexto = texto

# # Interação
# pergunta = input("Digite sua pergunta: ")
# resposta = qa_pipeline(question=pergunta, context=contexto)


# Pergunta simulada
pergunta = "Que tarefas a IA pode executar, segundo o texto?"
resposta = qa_pipeline(question=pergunta, context=contexto)


print("Pergunta:", pergunta)
print("Resposta:", resposta['answer'])


Device set to use cpu


Pergunta: Que tarefas a IA pode executar, segundo o texto?
Resposta: normalmente exigiriam inteligência humana


In [4]:
"""
Célula 5: Execução do agente de perguntas e respostas

Esta célula:
- Recebe uma pergunta do usuário
- Usa o modelo local 'distilbert-base-cased-distilled-squad' via Transformers
- Retorna a resposta com base no texto carregado
- Modelo com resposta a perguntas (QA) local, model="distilbert-base-cased-distilled-squad RESPOSTAS GENERICAS
"""

from transformers import pipeline

# Modelo alternativo mais objetivo
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

# Texto de contexto da célula 3
contexto = texto

# Pergunta simulada
pergunta = "Que tarefas a IA pode executar, segundo o texto?"

# Geração da resposta
resposta = qa_pipeline(question=pergunta, context=contexto)

print("Pergunta:", pergunta)
print("Resposta:", resposta['answer'])


Device set to use cpu


Pergunta: Que tarefas a IA pode executar, segundo o texto?
Resposta: normalmente exigiriam inteligência humana


In [5]:
"""
Célula 6: Execução do agente de perguntas e respostas

Esta célula:
- Recebe uma pergunta do usuário
- usar modelo generativo para perguntas e respostas
- Retorna a resposta com base no texto carregado
- Modelo com resposta a perguntas (QA) local, model=google/flan-t5-base RESPOSTAS MAIS COMPLETAS
- Usa o modelo google/flan-t5-base para gerar respostas completas com base no contexto.

Obs.: Treinado em inglês, mas pode responder em português
"""

from transformers import pipeline

# Pipeline de geração de texto com modelo T5
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base")

# Montar prompt com contexto
prompt = f"Liste as tarefas que a inteligência artificial pode executar, segundo o texto: {texto}"

# Gerar resposta
resposta = qa_pipeline(prompt, max_length=60, do_sample=False)

print("Pergunta:", prompt)
print("Resposta:", resposta[0]['generated_text'])


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Pergunta: Liste as tarefas que a inteligência artificial pode executar, segundo o texto: 
A inteligência artificial (IA) é o ramo da ciência da computação que se dedica a criar sistemas capazes de executar tarefas que normalmente exigiriam inteligência humana.
Essas tarefas incluem reconhecimento de fala, tomada de decisões, tradução de idiomas e muito mais.
Os agentes de IA utilizam modelos treinados em grandes quantidades de dados para realizar inferências e responder a comandos.

Resposta: List as tasks that artificial intelligence (AI) can execute, secondo o texto: Artificial intelligence (AI) é o ramo da ciência da computaço que se dedica a criar sistemas capazes de executar tasks that normally require human intelligence. These tasks include reconhecimento de fala, tomada decises, traducio de idiomas e muito mais. The agents of AI use three-dimensional models in large quantities of data to perform inferences and respond to commands.


In [6]:
"""
Célula 7: Execução do agente com modelo generativo (google/mt5-small)

Esta célula:
- Recebe uma pergunta do usuário como prompt
- Utiliza um modelo generativo para responder com base no texto carregado
- Emprega o modelo local 'google/mt5-small', que oferece suporte multilíngue, incluindo o português
- Gera respostas mais completas e contextuais com base no conteúdo fornecido
"""

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/mt5-small", use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Reformulado com o prefixo "perguntar:", compatível com o estilo MT5
prompt = f"perguntar: Quais tarefas a inteligência artificial pode executar de acordo com o texto abaixo?\n{texto}"
prompt = f"perguntar: O que é IA?\n{texto}"
# prompt = f"perguntar: Qual é a previsão do tempo para amanhã em São Paulo?\n{texto}"

# Lista de perguntas para testar o modelo
resposta = qa_pipeline(prompt, max_new_tokens=60, do_sample=False)

print(type(resposta))
print("Pergunta:", prompt)
print("Resposta:", resposta[0]['generated_text'])

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cpu


<class 'list'>
Pergunta: perguntar: O que é IA?

A inteligência artificial (IA) é o ramo da ciência da computação que se dedica a criar sistemas capazes de executar tarefas que normalmente exigiriam inteligência humana.
Essas tarefas incluem reconhecimento de fala, tomada de decisões, tradução de idiomas e muito mais.
Os agentes de IA utilizam modelos treinados em grandes quantidades de dados para realizar inferências e responder a comandos.

Resposta: <extra_id_0>.


In [7]:
"""
Célula 7: Execução do agente com modelo generativo (google/flan-t5-base)

Melhorias aplicadas:
- Texto base expandido
- Prompt reformulado com instruções claras
- Aumento do número de tokens para evitar corte da resposta
"""

import random
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Texto expandido
texto = """
A inteligência artificial (IA) é uma área da ciência da computação que desenvolve sistemas capazes de simular a inteligência humana.
Esses sistemas aprendem com dados, identificam padrões, tomam decisões e resolvem problemas de maneira autônoma ou assistida.
Entre as tarefas comuns da IA estão: reconhecimento de fala, tradução automática, análise de imagens, automação de processos,
assistentes virtuais e recomendação personalizada de conteúdos. A IA está presente em áreas como saúde, finanças, educação,
logística e atendimento ao cliente.
"""

# Dicionário de perguntas por categoria
perguntas = {
    "definicao": [
        "O que é inteligência artificial?",
        "Explique o conceito de IA.",
        "O que significa IA?"
    ],
    "tarefas": [
        "Quais tarefas a inteligência artificial pode executar?",
        "A IA pode reconhecer fala, traduzir idiomas e tomar decisões?",
        "Liste exemplos de atividades feitas por IA."
    ],
    "limites": [
        "Quais são as limitações da IA?",
        "A IA pode substituir completamente os humanos?",
        "A inteligência artificial pode tomar decisões sozinha?"
    ]
}

# Escolha aleatória
categoria = random.choice(list(perguntas.keys()))
pergunta = random.choice(perguntas[categoria])

# Prompt aprimorado
prompt = f"""
Com base no texto a seguir, responda à pergunta de forma clara, completa e original.
Texto:
\"\"\"{texto}\"\"\"

Pergunta:
{pergunta}

Responda:
"""

# Modelo e pipeline
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Geração
resposta = qa_pipeline(prompt, max_new_tokens=150, do_sample=False)

# Exibir
print("Categoria:", categoria)
print("Pergunta:", pergunta)
print("Resposta:", resposta[0]["generated_text"])


Device set to use cpu


Categoria: definicao
Pergunta: Explique o conceito de IA.
Resposta: "" A inteligência artificial (IA) é uma área da ciência da computaço que desenvolve sistemas capazes de simular inteligência humana. These systems aprendem com datos, identificam padres, tomam decises e resolvem problemas autônoma ou assistida. Entre as tarefas comuns de IA esto: reconhecimento de fala, traducio automática, análise de imagens, auto


In [8]:
"""
Célula 7: Execução do agente com modelo generativo (google/flan-t5-large)

Esta célula:
- Define um dicionário com perguntas agrupadas por categorias
- Seleciona aleatoriamente uma pergunta
- Utiliza o modelo local 'google/flan-t5-large', que oferece maior capacidade de compreensão
- Gera uma resposta com base no conteúdo carregado como texto base
"""

import random
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Texto fixo como base de conhecimento
texto = """
A inteligência artificial (IA) é o ramo da ciência da computação que se dedica a criar sistemas capazes de executar tarefas que normalmente exigiriam inteligência humana.
Essas tarefas incluem reconhecimento de fala, tomada de decisões, tradução de idiomas e muito mais.
Os agentes de IA utilizam modelos treinados em grandes quantidades de dados para realizar inferências e responder a comandos.
"""

# Dicionário de perguntas agrupadas por categorias
perguntas = {
    "definicao": [
        "O que é inteligência artificial?",
        "Explique o conceito de IA.",
        "O que significa IA?"
    ],
    "tarefas": [
        "Quais tarefas a inteligência artificial pode executar?",
        "A IA pode reconhecer fala, traduzir idiomas e tomar decisões?",
        "Liste exemplos de atividades feitas por IA."
    ],
    "limites": [
        "Quais são as limitações da IA?",
        "A IA pode substituir completamente os humanos?",
        "A inteligência artificial pode tomar decisões sozinha?"
    ]
}

# Selecionar uma pergunta aleatória
categoria = random.choice(list(perguntas.keys()))
pergunta = random.choice(perguntas[categoria])

# Reformular o prompt para melhor compreensão
prompt = f"Responda em português com detalhes: {pergunta}\n\nBase de conhecimento:\n{texto}"

# Carregar modelo e tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Gerar resposta com mais tokens
resposta = qa_pipeline(prompt, max_new_tokens=150, do_sample=False)

# Exibir resultado
print("Device set to use cpu")
print("Categoria:", categoria)
print("Pergunta:", pergunta)
print("Resposta:", resposta[0]['generated_text'].strip())


Device set to use cpu


Device set to use cpu
Categoria: definicao
Pergunta: O que significa IA?
Resposta: A inteligência artificial (IA) é o ramo da ciência da computaço que se dedica a criar sistemas capazes de executar tarefas que normalmente exigiriam inteligência humana.


In [2]:
import random
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# --- Texto fixo como base de conhecimento ---
texto_pt = """
A inteligência artificial (IA) é o ramo da ciência da computação que se dedica a criar sistemas capazes de executar tarefas que normalmente exigiriam inteligência humana.
Essas tarefas incluem reconhecimento de fala, tomada de decisões, tradução de idiomas e muito mais.
Os agentes de IA utilizam modelos treinados em grandes quantidades de dados para realizar inferências e responder a comandos.
"""

# --- Dicionário de perguntas ---
perguntas = {
    "definicao": [
        "O que é inteligência artificial?",
        "Explique o conceito de IA.",
        "O que significa IA?"
    ],
    "tarefas": [
        "Quais tarefas a inteligência artificial pode executar?",
        "A IA pode reconhecer fala, traduzir idiomas e tomar decisões?",
        "Liste exemplos de atividades feitas por IA."
    ],
    "limites": [
        "Quais são as limitações da IA?",
        "A IA pode substituir completamente os humanos?",
        "A inteligência artificial pode tomar decisões sozinha?"
    ]
}

# --- Seleção aleatória de pergunta ---
categoria = random.choice(list(perguntas.keys()))
pergunta_pt = random.choice(perguntas[categoria])

# --- Pipelines de tradução ---
# Instale opcionalmente: pip install sacremoses
tradutor_pt_en = pipeline("translation", model="manueldeprada/t5-small-pt-en")
tradutor_en_pt = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")

# Tradução da pergunta e texto
pergunta_en = tradutor_pt_en(pergunta_pt)[0]["translation_text"]
texto_en = tradutor_pt_en(texto_pt)[0]["translation_text"]

# --- Prompt para resposta ---
prompt = f"Question: {pergunta_en}\n\nContext:\n{texto_en}\n\nAnswer in English:"

# --- Carregando modelo de QA (Flan-T5) ---
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Geração da resposta (em inglês)
resposta_en = qa_pipeline(prompt, max_new_tokens=150, do_sample=False)[0]["generated_text"]

# Tradução da resposta para português
resposta_pt = tradutor_en_pt(resposta_en)[0]["translation_text"]

# --- Exibição do resultado final ---
print("Categoria:", categoria)
print("Pergunta:", pergunta_pt)
print("Resposta:", resposta_pt)


Device set to use cpu
Device set to use cpu
Device set to use cpu


Categoria: tarefas
Pergunta: A IA pode reconhecer fala, traduzir idiomas e tomar decisões?
Resposta: A inteligência artificial (IA) é o núcleo da ciência da computação que permite que os sistemas executem tarefas que normalmente exigem inteligência humana. Essas tarefas incluem o reconhecimento da fala, a tomada de decisões, a tradução de sons e muito mais.


### 📎 Anexo: Glossário de Termos Técnicos

Este glossário tem como objetivo auxiliar na compreensão dos principais conceitos utilizados no desenvolvimento do agente de IA presente neste notebook.

---

#### 🔹 Embeddings
Embeddings são representações vetoriais de textos, palavras ou sentenças em um espaço numérico de alta dimensão. Eles capturam o significado semântico dos dados e permitem comparar similaridades entre conteúdos diferentes. São fundamentais para buscas por similaridade em bases vetoriais.

---

#### 🔹 Vector Store
É uma base de dados especializada em armazenar vetores (como embeddings). Permite realizar buscas semânticas ao encontrar vetores semelhantes a um vetor de consulta. No projeto, o Chroma é utilizado como vector store.

---

#### 🔹 Prompt
Prompt é o texto de entrada enviado ao modelo de linguagem (LLM) para gerar uma resposta. Um bom prompt orienta o modelo a produzir respostas mais relevantes e específicas.

---

#### 🔹 Pipeline (HuggingFace)
Um pipeline é uma abstração que simplifica o uso de modelos da HuggingFace. Ele automatiza tarefas como tokenização, entrada e saída de dados e inferência do modelo. Exemplo: `pipeline("text2text-generation")` para gerar texto baseado em outro texto.

---

#### 🔹 LLM (Large Language Model)
Modelos de linguagem de grande porte treinados em vastos conjuntos de dados. São capazes de compreender e gerar linguagem humana de forma sofisticada. Exemplos: GPT, BERT, T5.

---
